In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import warnings
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Tabular Playground Analysis

# Read Data

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.drop(['id'], axis=1, inplace=True)


In [ ]:
train_df

In [ ]:
#Examine target value
train_df['target'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
colors=['#5e5e5e','#919191','#c6c6c6','#ffffff']
sns.set(palette=colors, style='white', rc={'axes.facecolor':'#1c465f', 'figure.facecolor':'#1c465f','text.color':'white','axes.labelcolor':'white'
                                                       , 'xtick.color':'white','ytick.color':'white', 'axes.edgecolor':'white',
                                          'font.family': ['serif'],'font.sans-serif': ['Liberation Sans']})
sns.palplot(colors)


In [ ]:
fig=plt.figure(figsize=(15,8))

ax=sns.countplot(train_df['target'], linewidth=2);
plt.title("Target variable counts", size='20', weight='bold')

for p in ax.patches:
        ax.annotate(p.get_height(), (p.get_x()+0.25, p.get_height()+1000), ha='left', size=15)
        


In [ ]:
train_df.describe().T.style.bar(subset=['std'])

In [ ]:
feat=train_df.std().nlargest(6).index.to_list()
feat

In [ ]:
from scipy.stats import norm
fig,ax=plt.subplots(nrows=2, ncols=3, figsize=(20,8), sharey=True)
plt.suptitle("Distribution of Feature with high Std.Dev", size=20, weight=20)
ax=ax.flatten()
for i,j in enumerate(feat):
    sns.kdeplot(train_df[j], ax=ax[i], fill=True, color='white', hue=train_df['target'])
    ax[i].axvline(train_df[j].mean(), ls='--', c='white')
    ax[i].set_xlim(ax[i].get_xlim()[0],20)

***In all the above high density features, Class 2 has last data points compared to other 3 classes. So, there is imbalance in data.***

In [ ]:
from scipy.stats import norm
fig,ax=plt.subplots(nrows=2, ncols=3, figsize=(20,8), sharey=True)
plt.suptitle("Distribution of Feature with high Std.Dev", size=20, weight=20)
ax=ax.flatten()
for i,j in enumerate(feat):
    sns.scatterplot(x=train_df[j],y=train_df['feature_35'], ax=ax[i], hue=train_df['target'])
    

*** couldn't find cluster or relationship in the data.***

In [ ]:
from scipy.stats import norm
fig,ax=plt.subplots(nrows=2, ncols=3, figsize=(20,8), sharey=True)
plt.suptitle("Distribution of Feature with high Std.Dev", size=20, weight=20)
ax=ax.flatten()
for i,j in enumerate(feat):
    sns.boxplot(x=train_df[j], ax=ax[i], y=train_df['target'])

***There are lot of outliers in the data***

In [ ]:
feat.append('target')


In [ ]:
sns.pairplot(train_df[feat], hue='target')

In [ ]:
sns.jointplot(x=train_df['feature_38'],y=train_df['feature_39'], kind='kde', hue=train_df['target'])

# Lets try to decompose the data so find patern

In [ ]:
#univariated analysis
class_v={'Class_1':0,'Class_2':1,'Class_3':2,'Class_4':3}
train_df['target']=train_df['target'].map(class_v)

In [ ]:
#train_df.drop('id', axis=1, inplace=True)
X=train_df.drop('target', axis=1)
y=train_df['target']
cols=[]
#X=X.apply(lambda x: np.log(x+1))
for i in X.columns:
    if X[i].nunique()<10:
        cols.append(i)

#from imblearn import over_sampling
#samp = over_sampling.SMOTE()
#X, y=samp.fit_resample(X,y)

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)
dep_cols = train_df.columns



from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, QuantileTransformer, PowerTransformer
scale = QuantileTransformer().fit(X_train.drop(cols, axis=1))
X_train_scale=scale.transform(X_train.drop(cols,axis=1))
X_test_scale=scale.transform(X_test.drop(cols, axis=1))



In [ ]:
cols

In [ ]:
import seaborn as sns
sns.kdeplot(X_train_scale[:,10])

In [ ]:
sns.scatterplot(x=X_train_decom[:,0],y=X_train_decom[:,1], hue=y_train)

***decomposition technique also doesn't provide pattern in data***

# Model creation

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
#from sklearn.svm import SVC
reg= LogisticRegressionCV(cv=5, scoring='accuracy')
#reg=SVC()
reg.fit(X_train_scale, y_train)
print(reg.score(X_train_scale, y_train))
pred=reg.predict(X_test_scale)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

***Simple logisitc regression give 57% accuracy but it always predicts Class 2. so we need to go for alternative method***

In [ ]:
from xgboost import XGBClassifier, plot_importance, XGBRFClassifier

X= train_df.drop(['target'], axis=1)
y=train_df['target']

#from imblearn import over_sampling
#samp = over_sampling.SMOTE(k_neighbors=40)
#X, y=samp.fit_resample(X,y)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)
dep_cols = train_df.columns

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, QuantileTransformer
scale = QuantileTransformer().fit(X_train.drop(cols, axis=1))
X_train_scale=scale.transform(X_train.drop(cols,axis=1))
X_test_scale=scale.transform(X_test.drop(cols, axis=1))


model1 = XGBClassifier(use_label_encoder=False)
model1.fit(X_train_scale,y_train)
print(model1.score(X_train_scale,y_train))
pred=model1.predict(X_test_scale)


In [ ]:
print(accuracy_score(y_test,pred))
print(classification_report(y_test, pred))
from sklearn.metrics import confusion_matrix
import seaborn as sns
sns.heatmap(confusion_matrix(y_test, pred), annot=True, fmt='2d')

In [ ]:
fig=plt.figure(figsize=(15,8))
plot_importance(model1)

In [ ]:
X1=scale.transform(X.drop(cols,axis=1))
model1.fit(X1,y)

In [ ]:
test_x=test_df.drop('id',axis=1)
test_x_scale=scale.transform(test_x.drop(cols,axis=1))
#test_x_scale.shape
#test_x_best=seq.transform(test_x_scale)
#print(test_x_best.shape)
pred_test=model1.predict_proba(test_x_scale)
pred_test = pd.DataFrame(pred_test, columns=['Class_1','Class_2','Class_3','Class_4'])
pred_test=pred_test.set_index(test_df.index)
df=pd.concat([test_df['id'],pred_test], ignore_index=False, axis=1)
df.to_csv('submission1.csv', index=False)

# Try with H2O.ai

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

In [ ]:
train = h2o.H2OFrame(train_df)
train['target']=train['target'].asfactor()
feat = train.drop(['target'], axis=1).columns
target='target'

In [ ]:
model = H2OAutoML(seed=13,stopping_metric='logloss',preprocessing=['target_encoding'],exploitation_ratio=0.1, sort_metric='logloss')
model.train(x=feat, y=target, training_frame=train)

In [ ]:
model.leaderboard

In [ ]:
test=h2o.H2OFrame(test_df)
pred = model.leader.predict(test.drop('id', axis=1))

In [ ]:
#df = pd.DataFrame(columns=['id','Class_1','Class_2','Class_3','Class_4'])
df=h2o.as_list(pred, use_pandas=True)

df['id']=test_df['id']

df.drop(['predict'], axis=1, inplace=True)
df=df[['id','Class_1','Class_2','Class_3','Class_4']]
df.to_csv('submission_automl.csv', index=False)

I will keep working to improve the score.